In [1]:
# bow vs tfidf

# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os
import dagshub

In [2]:
mlflow.set_tracking_uri('https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow')
dagshub.init(repo_owner='kunalkaushik1982', repo_name='womens-ecommerce-clothing-reviews', mlflow=True)

Accessing as kunalkaushik1982

Initialized MLflow to track repo "kunalkaushik1982/womens-ecommerce-clothing-reviews"

Repository kunalkaushik1982/womens-ecommerce-clothing-reviews initialized!

In [3]:
reviews=pd.read_csv(r'C:\Users\work\Documents\womens-ecommerce-clothing-reviews\Womens Clothing E-Commerce Reviews.csv')
reviews.drop(['Unnamed: 0', 'Clothing ID','Age','Title','Recommended IND','Positive Feedback Count','Division Name','Department Name','Class Name'], axis=1, inplace=True)
reviews=reviews.dropna(how='any',axis=0)
reviews.head()

,Review Text,Rating
0,Absolutely wonderful - silky and sexy and comf...,4
1,Love this dress! it's sooo pretty. i happene...,5
2,I had such high hopes for this dress and reall...,3
3,"I love, love, love this jumpsuit. it's fun, fl...",5
4,This shirt is very flattering to all due to th...,5


In [4]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\work\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\work\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['Review Text'] = df['Review Text'].apply(lower_case)
        df['Review Text'] = df['Review Text'].apply(remove_stop_words)
        df['Review Text'] = df['Review Text'].apply(removing_numbers)
        df['Review Text'] = df['Review Text'].apply(removing_punctuations)
        df['Review Text'] = df['Review Text'].apply(removing_urls)
        df['Review Text'] = df['Review Text'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
reviews = normalize_text(reviews)
reviews.head()

,Review Text,Rating
0,absolutely wonderful silky sexy comfortable,4
1,love dress sooo pretty happened find store i m...,5
2,high hope dress really wanted work me initiall...,3
3,love love love jumpsuit fun flirty fabulous ev...,5
4,shirt flattering due adjustable front tie perf...,5


In [7]:
# Set the experiment name
mlflow.set_experiment("Bow vs TfIdf")

2024/12/28 18:05:59 INFO mlflow.tracking.fluent: Experiment with name 'Bow vs TfIdf' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/be783a82d1454492b8f3dc109305b92a', creation_time=1735389359849, experiment_id='2', last_update_time=1735389359849, lifecycle_stage='active', name='Bow vs TfIdf', tags={}>

In [16]:
# Define feature extraction methods
vectorizers = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

# Define algorithms
algorithms = {
    'LogisticRegression': LogisticRegression(max_iter=1000),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

In [17]:
# Start the parent run
with mlflow.start_run(run_name="All Experiments") as parent_run:
    # Loop through algorithms and feature extraction methods (Child Runs)
    for algo_name, algorithm in algorithms.items():
        for vec_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                X = vectorizer.fit_transform(reviews['Review Text'])
                y = reviews['Rating']
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                

                # Log preprocessing parameters
                mlflow.log_param("vectorizer", vec_name)
                mlflow.log_param("algorithm", algo_name)
                mlflow.log_param("test_size", 0.2)
                
                # Model training
                model = algorithm
                if algo_name == 'XGBoost':
                    y_train=y_train-1
                    y_test = y_test - 1
                model.fit(X_train, y_train)
                
                # Log model parameters
                if algo_name == 'LogisticRegression':
                    mlflow.log_param("C", model.C)
                elif algo_name == 'MultinomialNB':
                    mlflow.log_param("alpha", model.alpha)
                elif algo_name == 'XGBoost':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                elif algo_name == 'RandomForest':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("max_depth", model.max_depth)
                elif algo_name == 'GradientBoosting':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                    mlflow.log_param("max_depth", model.max_depth)
                
                # Model evaluation
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred,average='weighted')
                recall = recall_score(y_test, y_pred,average='weighted')
                f1 = f1_score(y_test, y_pred,average='weighted')

                if algo_name == 'XGBoost':
                    y_pred=y_pred+1
                    
                
                # Log evaluation metrics
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1_score", f1)
                
                # Log model
                mlflow.sklearn.log_model(model, "model")

                 # Save and log the notebook
                import os
                notebook_path = "exp3_bow_vs_tfidf.ipynb"
                os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
                mlflow.log_artifact(notebook_path)
                                
                # Print the results for verification
                print(f"Algorithm: {algo_name}, Feature Engineering: {vec_name}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1}")

2024/12/28 18:31:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: LogisticRegression, Feature Engineering: BoW
Accuracy: 0.6224332082137337
Precision: 0.5943691472538253
Recall: 0.6224332082137337
F1 Score: 0.6053411013293883
🏃 View run LogisticRegression with BoW at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/2/runs/9e069ceccec7403b891260d6849bdbd0
🧪 View experiment at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/2


2024/12/28 18:32:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: LogisticRegression, Feature Engineering: TF-IDF
Accuracy: 0.6400971516891146
Precision: 0.5916707828316871
Recall: 0.6400971516891146
F1 Score: 0.6030011306259877
🏃 View run LogisticRegression with TF-IDF at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/2/runs/f2e5942ca4ff40ef830656f13f6b5223
🧪 View experiment at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/2


2024/12/28 18:32:14 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

Algorithm: RandomForest, Feature Engineering: TF-IDF
Accuracy: 0.5837933318613381
Precision: 0.4912022881283569
Recall: 0.5837933318613381
F1 Score: 0.46648171397362875
🏃 View run RandomForest with TF-IDF at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/2/runs/a165bdd2316243d0840f02c6e5958403
🧪 View experiment at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/2


2024/12/28 18:39:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: GradientBoosting, Feature Engineering: BoW
Accuracy: 0.6056524619121219
Precision: 0.5398211798187565
Recall: 0.6056524619121219
F1 Score: 0.5314447426266099
🏃 View run GradientBoosting with BoW at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/2/runs/e8a6316aee61477aa114194da43e8d7a
🧪 View experiment at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/2


2024/12/28 18:41:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Algorithm: GradientBoosting, Feature Engineering: TF-IDF
Accuracy: 0.6100684477809671
Precision: 0.5425123766690234
Recall: 0.6100684477809671
F1 Score: 0.5374018263695727
🏃 View run GradientBoosting with TF-IDF at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/2/runs/90fdd505b99742eb82125e7c57e5ee42
🧪 View experiment at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/2
🏃 View run All Experiments at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/2/runs/dded25a19d7043e59f8943c6599dcee8
🧪 View experiment at: https://dagshub.com/kunalkaushik1982/womens-ecommerce-clothing-reviews.mlflow/#/experiments/2
